<a href="https://colab.research.google.com/github/brew-brew-com/ML-Prep/blob/main/12_Regression_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1>勾配ブースティング（Gradient Boosting Reggressor）で学習誤差と汎化誤差を算出し交差検証（Cross Validation）を実施する</h1> 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score

In [ ]:
boston = load_boston()

X = boston.data
y = boston.target

X_scl = StandardScaler().fit_transform(X) # データセットの標準化
model = GradientBoostingRegressor().fit(X_scl, y) # 標準化データセットからGradient Boostでfit
y_pred = model.predict(X_scl) # fitしたモデルで説明変数から目的変数を予測

# 目的変数と説明変数から予測したデータでR2スコアを算出。
# 学習した既知のデータセットを使っているだけなので、これは【学習誤差】を見ていることになる。

r2 = r2_score(y, y_pred)
print(r2)

0.9761405838418584


In [ ]:
# Pipilineを使って上記と同じことを行う
boston = load_boston()

X = boston.data
y = boston.target

pipe = Pipeline([("scl", StandardScaler()), ("gbr", GradientBoostingRegressor())])
model = pipe.fit(X, y)
y_pred = model.predict(X)

r2 = r2_score(y, y_pred)
print(r2)

0.9761405838418584


In [ ]:
boston = load_boston()

X = boston.data
y = boston.target

# データセットを学習データ（80%）とテストデータ（20%）に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

pipe = Pipeline([("scl", StandardScaler()), ("gbr", GradientBoostingRegressor())])
model = pipe.fit(X_train, y_train) # 学習用データセットでモデルを作成
y_pred = model.predict(X_test) # 学習済みモデルにテスト用の説明変数を読み込ませ、目的変数を予測

# 目的変数と説明変数から予測したデータでR2スコアを算出。
# 未知のテスト用データセットを使っているので、これは【汎化誤差】を見ていることになる。

r2 = r2_score(y_test, y_pred)
print(r2)

0.9090635145337592


In [ ]:
# パラメーター違いのCross Validation（交差検証）
# ランダム分割するだけの単純な交差検証は【Hold-out】と呼ばれる

boston = load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

pipe = Pipeline([("scl", StandardScaler()), ("gbr", GradientBoostingRegressor(max_depth=3,random_state=1))])
model = pipe.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print('-----------------------------------------------------')
print("■GBR（max_depth=3,random_state=1）")
print("Learinig Errorr（学習誤差）：" + str(r2_train))
print("Generalization Errorr（汎化誤差）：" + str(r2_test))
print('-----------------------------------------------------')

model = Pipeline([("scl", StandardScaler()), ("gbr", GradientBoostingRegressor(max_depth=10,random_state=1))]).fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print("■GBR（max_depth=10,random_state=1）")
print("Learinig Errorr（学習誤差）：" + str(r2_train))
print("Generalization Errorr（汎化誤差）：" + str(r2_test))
print('-----------------------------------------------------')

# => ■GBR（max_depth=3,random_state=1）
# => Learinig Errorr（学習誤差）：0.978281908615733
# => Generalization Errorr（汎化誤差）：0.9322914245170626

# => ■GBR（max_depth=10,random_state=1）
# => Learinig Errorr（学習誤差）：0.9999999896786709
# => Generalization Errorr（汎化誤差）：0.8648229443372307

# (max_depth=10,random_state=1)は【学習誤差】が1に近いが【汎化誤差】の値が低い。これはOvertfitting（過学習）であると判断できる。

-----------------------------------------------------
■GBR（max_depth=3,random_state=1）
Learinig Errorr（学習誤差）：0.9781572191469993
Generalization Errorr（汎化誤差）：0.93943156981112
-----------------------------------------------------
■GBR（max_depth=10,random_state=1）
Learinig Errorr（学習誤差）：0.9999999759812008
Generalization Errorr（汎化誤差）：0.9332346318138681
-----------------------------------------------------


In [ ]:
# 各種アルゴリズムをまとめてドン

boston = load_boston()

X = boston.data
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

pipelines = {
    "Ordinary Least Squares:\t" : Pipeline([("scl", StandardScaler()), ("est", LinearRegression())]),
    "Ridge:\t" : Pipeline([("scl", StandardScaler()), ("est", Ridge(random_state=0))]),
    "Random Forest:\t" : Pipeline([("scl", StandardScaler()), ("est", RandomForestRegressor(random_state=0))]),
    "Gradient Boosting:\t" : Pipeline([("scl", StandardScaler()), ("est", GradientBoostingRegressor(random_state=0))])
}

print('-----------------------------------------------------')
for pipe_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    print(pipe_name, r2_score(y_test, pipeline.predict(X_test)))
print('-----------------------------------------------------')

-----------------------------------------------------
Ordinary Least Squares:	 0.7634174432138472
Ridge:	 0.7634038058032349
Random Forest:	 0.9104769072940039
Gradient Boosting:	 0.9248353805926237
-----------------------------------------------------


In [ ]:
# ついでに最小2乗回帰（OLS：Ordinary Least Squares Regression）とリッジ回帰（Ridge Regression）のリニア系でもCross Validation

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

model_linear = Pipeline([('scl', StandardScaler()), ('est', LinearRegression())]).fit(X_train, y_train)
model_ridge = Pipeline([('scl', StandardScaler()), ('est', Ridge(alpha=10.0))]).fit(X_train, y_train)

# 性能指標の表示
print('-----------------------------------------------------')
print('Test Score of OLS:\t %.6f'% r2_score(y_test, model_linear.predict(X_test)))
print('Test Score of Ridge:\t %.6f'% r2_score(y_test, model_ridge.predict(X_test)))

# 回帰係数の総和比較。アンサンブル系ではこれはできない。
# リッジ回帰の正則化項の役割把握のため（モデルの性能評価ではありません）
print('-----------------------------------------------------')
print('Absolute Sum of coefficient of OLS:\t %.6f'%(np.absolute(model_linear.named_steps['est'].coef_).sum()))
print('Absolute Sum of coefficient of Ridge:\t %.6f'%(np.absolute(model_ridge.named_steps['est'].coef_).sum()))

print('-----------------------------------------------------')

-----------------------------------------------------
Test Score of OLS:	 0.763417
Test Score of Ridge:	 0.761650
-----------------------------------------------------
Absolute Sum of coefficient of OLS:	 22.063408
Absolute Sum of coefficient of Ridge:	 19.487380
-----------------------------------------------------
